In [1]:
# !pip install tabulate

In [2]:
import pandas as pd
import numpy as np
import math
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm


In [3]:
df = pd.read_csv('X_train_Hi5.csv').set_index('row_index')
df.head()

/tmp/ipykernel_1027/2198212355.py:1: DtypeWarning: Columns (108,110,111,115,117,118,122,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('X_train_Hi5.csv').set_index('row_index')


,piezo_station_department_code,piezo_station_update_date,piezo_station_investigation_depth,piezo_station_department_name,piezo_station_commune_code_insee,piezo_station_pe_label,piezo_station_bdlisa_codes,piezo_station_altitude,piezo_station_bss_code,piezo_station_commune_name,...,prelev_longitude_2,prelev_latitude_2,prelev_commune_code_insee_2,prelev_other_volume_sum,insee_%_agri,insee_pop_commune,insee_med_living_level,insee_%_ind,insee_%_const,piezo_groundwater_level_category
row_index,,,,,,,,,,,,,,,,,,,,,
0,01,Sun Jul 14 13:00:02 CEST 2024,20.00,Ain,01073,PIEZOMETRE - MARAIS DE LAVOURS (CEYZERIEU - BR...,['712AH37'],232.00,07004X0046/D6-20,Ceyzérieu,...,5.698947,45.725106,1454.0,1.793055e+09,11.8,992.0,25250,2.9,16.2,High
1,01,Sun Jul 14 13:00:02 CEST 2024,35.60,Ain,01363,PIEZOMETRE - GRAVIERE (ST-JEAN-LE-VIEUX - BRGM...,['712GB05'],247.25,06754X0077/F1,Saint-Jean-le-Vieux,...,5.464933,46.210734,1051.0,1.085125e+09,0.6,1786.0,24660,44.5,11,Very High
2,01,Sun Jul 14 13:00:02 CEST 2024,35.22,Ain,01244,PIEZOMETRE - BORD AUTOROUTE (MEXIMIEUX - BRGM ...,['040AJ43'],218.77,06993X0226/MEXI_2,Meximieux,...,5.085060,45.812828,69266.0,3.810492e+08,0,8085.0,24890,8.4,7.8,High
3,01,Sun Jul 14 13:00:02 CEST 2024,34.20,Ain,01288,PIEZOMETRE - GRENY (PERON - BRGM 01) - BSH,"['516AA00', '516AF00']",499.85,06533X0070/F2,Péron,...,5.802841,46.366049,39286.0,3.800911e+08,1.5,2838.0,39700,2.4,5.2,Very High
4,01,Sun Jul 14 13:00:02 CEST 2024,37.30,Ain,01422,FORAGE - ENCLOS (TOSSIAT - BRGM 01) - BSH,['507AB00'],260.00,06518X0026/P2,Tossiat,...,5.377265,46.080989,1273.0,1.966631e+07,0.2,1352.0,26180,21.5,9.8,Very Low


In [4]:
df.memory_usage().sum() / 1e9

np.float64(3.079383808)

In [5]:
# df.dtypes

In [6]:
to_drop = ['piezo_station_update_date',
           'piezo_station_department_name',
           'piezo_station_commune_code_insee',
           'piezo_station_pe_label',
           'piezo_station_bdlisa_codes',
           'piezo_station_commune_name',
           'piezo_station_bss_id',
           'piezo_bss_code',
           'piezo_continuity_name',
           'piezo_producer_name',
           'piezo_measure_nature_name', 
           'meteo_date',
           'meteo_id',
           'meteo_name',
           'hydro_station_code',
           'hydro_observation_date_elab',
           'hydro_status_label',
           'hydro_method_code',
           'hydro_method_label',
           'hydro_qualification_label',
           'prelev_structure_code_0',
           'prelev_commune_code_insee_0',
           'prelev_structure_code_1',
           'prelev_commune_code_insee_1',
           'prelev_structure_code_2',
           'prelev_commune_code_insee_2']

In [7]:
df = df.drop(to_drop, axis=1)
df.head()

,piezo_station_department_code,piezo_station_investigation_depth,piezo_station_altitude,piezo_station_bss_code,piezo_station_longitude,piezo_station_latitude,piezo_measurement_date,piezo_obtention_mode,piezo_status,piezo_qualification,...,prelev_volume_obtention_mode_label_2,prelev_longitude_2,prelev_latitude_2,prelev_other_volume_sum,insee_%_agri,insee_pop_commune,insee_med_living_level,insee_%_ind,insee_%_const,piezo_groundwater_level_category
row_index,,,,,,,,,,,,,,,,,,,,,
0,01,20.00,232.00,07004X0046/D6-20,5.748241,45.828148,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,...,Mesure indirecte,5.698947,45.725106,1.793055e+09,11.8,992.0,25250,2.9,16.2,High
1,01,35.60,247.25,06754X0077/F1,5.356637,46.028102,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,...,Mesure indirecte,5.464933,46.210734,1.085125e+09,0.6,1786.0,24660,44.5,11,Very High
2,01,35.22,218.77,06993X0226/MEXI_2,5.220795,45.895734,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,...,Mesure indirecte,5.085060,45.812828,3.810492e+08,0,8085.0,24890,8.4,7.8,High
3,01,34.20,499.85,06533X0070/F2,5.948977,46.201180,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,...,Mesure indirecte,5.802841,46.366049,3.800911e+08,1.5,2838.0,39700,2.4,5.2,Very High
4,01,37.30,260.00,06518X0026/P2,5.313353,46.136402,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,...,Mesure indirecte,5.377265,46.080989,1.966631e+07,0.2,1352.0,26180,21.5,9.8,Very Low


In [8]:
# dropper les duplicates mais on devra faire attention pour la prédiction

In [9]:
df.shape

(2830316, 109)

In [10]:
df = df.drop_duplicates(subset=['piezo_station_bss_code', 'piezo_measurement_date'])
df.shape

(2830072, 109)

# Réduction de la taille du jeu

In [11]:
# stations_subset = np.random.choice(df['piezo_station_bss_code'].unique(), size=500, replace=False)
stations_subset = list(df['piezo_station_bss_code'].unique())

In [12]:
df = df[df['piezo_station_bss_code'].isin(stations_subset)]
df

,piezo_station_department_code,piezo_station_investigation_depth,piezo_station_altitude,piezo_station_bss_code,piezo_station_longitude,piezo_station_latitude,piezo_measurement_date,piezo_obtention_mode,piezo_status,piezo_qualification,...,prelev_volume_obtention_mode_label_2,prelev_longitude_2,prelev_latitude_2,prelev_other_volume_sum,insee_%_agri,insee_pop_commune,insee_med_living_level,insee_%_ind,insee_%_const,piezo_groundwater_level_category
row_index,,,,,,,,,,,,,,,,,,,,,
0,01,20.00,232.00,07004X0046/D6-20,5.748241,45.828148,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,...,Mesure indirecte,5.698947,45.725106,1.793055e+09,11.8,992.0,25250,2.9,16.2,High
1,01,35.60,247.25,06754X0077/F1,5.356637,46.028102,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,...,Mesure indirecte,5.464933,46.210734,1.085125e+09,0.6,1786.0,24660,44.5,11,Very High
2,01,35.22,218.77,06993X0226/MEXI_2,5.220795,45.895734,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,...,Mesure indirecte,5.085060,45.812828,3.810492e+08,0,8085.0,24890,8.4,7.8,High
3,01,34.20,499.85,06533X0070/F2,5.948977,46.201180,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,...,Mesure indirecte,5.802841,46.366049,3.800911e+08,1.5,2838.0,39700,2.4,5.2,Very High
4,01,37.30,260.00,06518X0026/P2,5.313353,46.136402,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,...,Mesure indirecte,5.377265,46.080989,1.966631e+07,0.2,1352.0,26180,21.5,9.8,Very Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294080,95,24.00,63.00,01258X0020/S1,1.711584,49.197517,2023-05-31,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,...,NaN,NaN,NaN,NaN,27.3,322.0,25850,0,27.3,Very Low
3294081,95,25.50,103.00,01266X1013/S1,1.901276,49.145695,2023-05-31,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,...,NaN,NaN,NaN,NaN,1.1,2028.0,25320,13.8,2.9,Very Low
3294082,95,30.00,83.00,01522X0069/PZ2008,1.897576,49.083846,2023-05-31,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,...,NaN,NaN,NaN,NaN,1.6,295.0,31340,4.8,4.8,Low


# Nettoyage

## Piezo

In [13]:
df['piezo_station_department_code'].isna().mean()

np.float64(0.0)

In [14]:
one_hot_dummies = pd.get_dummies(df['piezo_station_department_code'], prefix='piezo_station_department_code')
# Combining the one-hot encoded columns with the original DataFrame
df_cleaned = pd.concat([df.drop('piezo_station_department_code', axis=1), one_hot_dummies], axis=1)
df_cleaned

,piezo_station_investigation_depth,piezo_station_altitude,piezo_station_bss_code,piezo_station_longitude,piezo_station_latitude,piezo_measurement_date,piezo_obtention_mode,piezo_status,piezo_qualification,piezo_continuity_code,...,piezo_station_department_code_84,piezo_station_department_code_85,piezo_station_department_code_86,piezo_station_department_code_87,piezo_station_department_code_88,piezo_station_department_code_89,piezo_station_department_code_90,piezo_station_department_code_91,piezo_station_department_code_93,piezo_station_department_code_95
row_index,,,,,,,,,,,,,,,,,,,,,
0,20.00,232.00,07004X0046/D6-20,5.748241,45.828148,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,False,False,False,False,False,False,False,False,False,False
1,35.60,247.25,06754X0077/F1,5.356637,46.028102,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,False,False,False,False,False,False,False,False,False,False
2,35.22,218.77,06993X0226/MEXI_2,5.220795,45.895734,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,False,False,False,False,False,False,False,False,False,False
3,34.20,499.85,06533X0070/F2,5.948977,46.201180,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,False,False,False,False,False,False,False,False,False,False
4,37.30,260.00,06518X0026/P2,5.313353,46.136402,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294080,24.00,63.00,01258X0020/S1,1.711584,49.197517,2023-05-31,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,False,False,False,False,False,False,False,False,False,True
3294081,25.50,103.00,01266X1013/S1,1.901276,49.145695,2023-05-31,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,False,False,False,False,False,False,False,False,False,True
3294082,30.00,83.00,01522X0069/PZ2008,1.897576,49.083846,2023-05-31,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,False,False,False,False,False,False,False,False,False,True


In [15]:
df['piezo_station_investigation_depth'].isna().mean()

np.float64(0.06327118179325474)

In [16]:
df['piezo_station_altitude'].isna().mean()

np.float64(0.0)

In [17]:
df['piezo_station_longitude'].isna().mean()

np.float64(0.0)

In [18]:
df['piezo_station_latitude'].isna().mean()

np.float64(0.0)

In [19]:
df['piezo_measurement_date'].isna().mean()

np.float64(0.0)

In [20]:
df_cleaned['piezo_measurement_date'] = pd.to_datetime(df['piezo_measurement_date'])

In [21]:
# Année
# Mois
# Jour de la semaine
# Tranche horaire
df_cleaned['piezo_measurement_date_year'] = df_cleaned['piezo_measurement_date'].dt.year
df_cleaned['piezo_measurement_date_month'] = df_cleaned['piezo_measurement_date'].dt.month
df_cleaned['piezo_measurement_date_day_of_week'] = df_cleaned['piezo_measurement_date'].dt.dayofweek  # 0 = Monday, 6 = Sunday


In [22]:
one_hot_dummies = pd.get_dummies(df_cleaned['piezo_measurement_date_year'], prefix='piezo_measurement_date_year')
# Combining the one-hot encoded columns with the original DataFrame
df_cleaned = pd.concat([df_cleaned.drop('piezo_measurement_date_year', axis=1), one_hot_dummies], axis=1)
df_cleaned

,piezo_station_investigation_depth,piezo_station_altitude,piezo_station_bss_code,piezo_station_longitude,piezo_station_latitude,piezo_measurement_date,piezo_obtention_mode,piezo_status,piezo_qualification,piezo_continuity_code,...,piezo_station_department_code_90,piezo_station_department_code_91,piezo_station_department_code_93,piezo_station_department_code_95,piezo_measurement_date_month,piezo_measurement_date_day_of_week,piezo_measurement_date_year_2020,piezo_measurement_date_year_2021,piezo_measurement_date_year_2022,piezo_measurement_date_year_2023
row_index,,,,,,,,,,,,,,,,,,,,,
0,20.00,232.00,07004X0046/D6-20,5.748241,45.828148,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,False,False,False,False,1,2,True,False,False,False
1,35.60,247.25,06754X0077/F1,5.356637,46.028102,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,False,False,False,False,1,2,True,False,False,False
2,35.22,218.77,06993X0226/MEXI_2,5.220795,45.895734,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,False,False,False,False,1,2,True,False,False,False
3,34.20,499.85,06533X0070/F2,5.948977,46.201180,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,False,False,False,False,1,2,True,False,False,False
4,37.30,260.00,06518X0026/P2,5.313353,46.136402,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,False,False,False,False,1,2,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294080,24.00,63.00,01258X0020/S1,1.711584,49.197517,2023-05-31,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,False,False,False,True,5,2,False,False,False,True
3294081,25.50,103.00,01266X1013/S1,1.901276,49.145695,2023-05-31,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,False,False,False,True,5,2,False,False,False,True
3294082,30.00,83.00,01522X0069/PZ2008,1.897576,49.083846,2023-05-31,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,False,False,False,True,5,2,False,False,False,True


In [23]:
one_hot_dummies = pd.get_dummies(df_cleaned['piezo_measurement_date_month'], prefix='piezo_measurement_date_month')
# Combining the one-hot encoded columns with the original DataFrame
df_cleaned = pd.concat([df_cleaned.drop('piezo_measurement_date_month', axis=1), one_hot_dummies], axis=1)
df_cleaned

,piezo_station_investigation_depth,piezo_station_altitude,piezo_station_bss_code,piezo_station_longitude,piezo_station_latitude,piezo_measurement_date,piezo_obtention_mode,piezo_status,piezo_qualification,piezo_continuity_code,...,piezo_measurement_date_month_3,piezo_measurement_date_month_4,piezo_measurement_date_month_5,piezo_measurement_date_month_6,piezo_measurement_date_month_7,piezo_measurement_date_month_8,piezo_measurement_date_month_9,piezo_measurement_date_month_10,piezo_measurement_date_month_11,piezo_measurement_date_month_12
row_index,,,,,,,,,,,,,,,,,,,,,
0,20.00,232.00,07004X0046/D6-20,5.748241,45.828148,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,False,False,False,False,False,False,False,False,False,False
1,35.60,247.25,06754X0077/F1,5.356637,46.028102,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,False,False,False,False,False,False,False,False,False,False
2,35.22,218.77,06993X0226/MEXI_2,5.220795,45.895734,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,False,False,False,False,False,False,False,False,False,False
3,34.20,499.85,06533X0070/F2,5.948977,46.201180,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,False,False,False,False,False,False,False,False,False,False
4,37.30,260.00,06518X0026/P2,5.313353,46.136402,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294080,24.00,63.00,01258X0020/S1,1.711584,49.197517,2023-05-31,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,False,False,True,False,False,False,False,False,False,False
3294081,25.50,103.00,01266X1013/S1,1.901276,49.145695,2023-05-31,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,False,False,True,False,False,False,False,False,False,False
3294082,30.00,83.00,01522X0069/PZ2008,1.897576,49.083846,2023-05-31,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,False,False,True,False,False,False,False,False,False,False


In [24]:
one_hot_dummies = pd.get_dummies(df_cleaned['piezo_measurement_date_day_of_week'], prefix='piezo_measurement_date_day_of_week')
# Combining the one-hot encoded columns with the original DataFrame
df_cleaned = pd.concat([df_cleaned.drop('piezo_measurement_date_day_of_week', axis=1), one_hot_dummies], axis=1)
df_cleaned

,piezo_station_investigation_depth,piezo_station_altitude,piezo_station_bss_code,piezo_station_longitude,piezo_station_latitude,piezo_measurement_date,piezo_obtention_mode,piezo_status,piezo_qualification,piezo_continuity_code,...,piezo_measurement_date_month_10,piezo_measurement_date_month_11,piezo_measurement_date_month_12,piezo_measurement_date_day_of_week_0,piezo_measurement_date_day_of_week_1,piezo_measurement_date_day_of_week_2,piezo_measurement_date_day_of_week_3,piezo_measurement_date_day_of_week_4,piezo_measurement_date_day_of_week_5,piezo_measurement_date_day_of_week_6
row_index,,,,,,,,,,,,,,,,,,,,,
0,20.00,232.00,07004X0046/D6-20,5.748241,45.828148,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,False,False,False,False,False,True,False,False,False,False
1,35.60,247.25,06754X0077/F1,5.356637,46.028102,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,False,False,False,False,False,True,False,False,False,False
2,35.22,218.77,06993X0226/MEXI_2,5.220795,45.895734,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,False,False,False,False,False,True,False,False,False,False
3,34.20,499.85,06533X0070/F2,5.948977,46.201180,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,False,False,False,False,False,True,False,False,False,False
4,37.30,260.00,06518X0026/P2,5.313353,46.136402,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294080,24.00,63.00,01258X0020/S1,1.711584,49.197517,2023-05-31,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,False,False,False,False,False,True,False,False,False,False
3294081,25.50,103.00,01266X1013/S1,1.901276,49.145695,2023-05-31,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,False,False,False,False,False,True,False,False,False,False
3294082,30.00,83.00,01522X0069/PZ2008,1.897576,49.083846,2023-05-31,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,False,False,False,False,False,True,False,False,False,False


In [25]:
df['piezo_obtention_mode'].isna().mean()

np.float64(0.0)

In [26]:
df['piezo_obtention_mode'].value_counts()

piezo_obtention_mode
Valeur mesurée              2657844
Mode d'obtention inconnu     138286
Valeur reconstituée           33942
Name: count, dtype: int64

In [27]:
one_hot_dummies = pd.get_dummies(df['piezo_obtention_mode'], prefix='piezo_obtention_mode')
# Combining the one-hot encoded columns with the original DataFrame
df_cleaned = pd.concat([df_cleaned.drop('piezo_obtention_mode', axis=1), one_hot_dummies], axis=1)
df_cleaned

,piezo_station_investigation_depth,piezo_station_altitude,piezo_station_bss_code,piezo_station_longitude,piezo_station_latitude,piezo_measurement_date,piezo_status,piezo_qualification,piezo_continuity_code,piezo_producer_code,...,piezo_measurement_date_day_of_week_0,piezo_measurement_date_day_of_week_1,piezo_measurement_date_day_of_week_2,piezo_measurement_date_day_of_week_3,piezo_measurement_date_day_of_week_4,piezo_measurement_date_day_of_week_5,piezo_measurement_date_day_of_week_6,piezo_obtention_mode_Mode d'obtention inconnu,piezo_obtention_mode_Valeur mesurée,piezo_obtention_mode_Valeur reconstituée
row_index,,,,,,,,,,,,,,,,,,,,,
0,20.00,232.00,07004X0046/D6-20,5.748241,45.828148,2020-01-01,Donnée contrôlée niveau 2,Correcte,2.0,196.0,...,False,False,True,False,False,False,False,False,True,False
1,35.60,247.25,06754X0077/F1,5.356637,46.028102,2020-01-01,Donnée contrôlée niveau 2,Correcte,2.0,196.0,...,False,False,True,False,False,False,False,False,True,False
2,35.22,218.77,06993X0226/MEXI_2,5.220795,45.895734,2020-01-01,Donnée contrôlée niveau 2,Correcte,2.0,196.0,...,False,False,True,False,False,False,False,False,True,False
3,34.20,499.85,06533X0070/F2,5.948977,46.201180,2020-01-01,Donnée contrôlée niveau 2,Correcte,2.0,196.0,...,False,False,True,False,False,False,False,False,True,False
4,37.30,260.00,06518X0026/P2,5.313353,46.136402,2020-01-01,Donnée contrôlée niveau 2,Correcte,2.0,196.0,...,False,False,True,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294080,24.00,63.00,01258X0020/S1,1.711584,49.197517,2023-05-31,Donnée contrôlée niveau 2,Correcte,2.0,238.0,...,False,False,True,False,False,False,False,False,True,False
3294081,25.50,103.00,01266X1013/S1,1.901276,49.145695,2023-05-31,Donnée contrôlée niveau 2,Correcte,2.0,291.0,...,False,False,True,False,False,False,False,False,True,False
3294082,30.00,83.00,01522X0069/PZ2008,1.897576,49.083846,2023-05-31,Donnée contrôlée niveau 2,Correcte,2.0,291.0,...,False,False,True,False,False,False,False,False,True,False


In [28]:
df['piezo_status'].isna().mean()

np.float64(0.0)

In [29]:
df['piezo_status'].value_counts()

piezo_status
Donnée contrôlée niveau 2    2146896
Donnée contrôlée niveau 1     564089
Donnée brute                   93199
Donnée interprétée             25888
Name: count, dtype: int64

In [30]:
one_hot_dummies = pd.get_dummies(df['piezo_status'], prefix='piezo_status')
# Combining the one-hot encoded columns with the original DataFrame
df_cleaned = pd.concat([df_cleaned.drop('piezo_status', axis=1), one_hot_dummies], axis=1)
df_cleaned

,piezo_station_investigation_depth,piezo_station_altitude,piezo_station_bss_code,piezo_station_longitude,piezo_station_latitude,piezo_measurement_date,piezo_qualification,piezo_continuity_code,piezo_producer_code,piezo_measure_nature_code,...,piezo_measurement_date_day_of_week_4,piezo_measurement_date_day_of_week_5,piezo_measurement_date_day_of_week_6,piezo_obtention_mode_Mode d'obtention inconnu,piezo_obtention_mode_Valeur mesurée,piezo_obtention_mode_Valeur reconstituée,piezo_status_Donnée brute,piezo_status_Donnée contrôlée niveau 1,piezo_status_Donnée contrôlée niveau 2,piezo_status_Donnée interprétée
row_index,,,,,,,,,,,,,,,,,,,,,
0,20.00,232.00,07004X0046/D6-20,5.748241,45.828148,2020-01-01,Correcte,2.0,196.0,N,...,False,False,False,False,True,False,False,False,True,False
1,35.60,247.25,06754X0077/F1,5.356637,46.028102,2020-01-01,Correcte,2.0,196.0,N,...,False,False,False,False,True,False,False,False,True,False
2,35.22,218.77,06993X0226/MEXI_2,5.220795,45.895734,2020-01-01,Correcte,2.0,196.0,N,...,False,False,False,False,True,False,False,False,True,False
3,34.20,499.85,06533X0070/F2,5.948977,46.201180,2020-01-01,Correcte,2.0,196.0,N,...,False,False,False,False,True,False,False,False,True,False
4,37.30,260.00,06518X0026/P2,5.313353,46.136402,2020-01-01,Correcte,2.0,196.0,N,...,False,False,False,False,True,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294080,24.00,63.00,01258X0020/S1,1.711584,49.197517,2023-05-31,Correcte,2.0,238.0,N,...,False,False,False,False,True,False,False,False,True,False
3294081,25.50,103.00,01266X1013/S1,1.901276,49.145695,2023-05-31,Correcte,2.0,291.0,N,...,False,False,False,False,True,False,False,False,True,False
3294082,30.00,83.00,01522X0069/PZ2008,1.897576,49.083846,2023-05-31,Correcte,2.0,291.0,N,...,False,False,False,False,True,False,False,False,True,False


In [31]:
df['piezo_qualification'].isna().mean()

np.float64(0.0)

In [32]:
df['piezo_qualification'].value_counts()

piezo_qualification
Correcte        2718625
Non qualifié      93199
Incorrecte         9462
Incertaine         8786
Name: count, dtype: int64

In [33]:
one_hot_dummies = pd.get_dummies(df['piezo_qualification'], prefix='piezo_qualification')
# Combining the one-hot encoded columns with the original DataFrame
df_cleaned = pd.concat([df_cleaned.drop('piezo_qualification', axis=1), one_hot_dummies], axis=1)
df_cleaned

,piezo_station_investigation_depth,piezo_station_altitude,piezo_station_bss_code,piezo_station_longitude,piezo_station_latitude,piezo_measurement_date,piezo_continuity_code,piezo_producer_code,piezo_measure_nature_code,meteo_latitude,...,piezo_obtention_mode_Valeur mesurée,piezo_obtention_mode_Valeur reconstituée,piezo_status_Donnée brute,piezo_status_Donnée contrôlée niveau 1,piezo_status_Donnée contrôlée niveau 2,piezo_status_Donnée interprétée,piezo_qualification_Correcte,piezo_qualification_Incertaine,piezo_qualification_Incorrecte,piezo_qualification_Non qualifié
row_index,,,,,,,,,,,,,,,,,,,,,
0,20.00,232.00,07004X0046/D6-20,5.748241,45.828148,2020-01-01,2.0,196.0,N,5.688000,...,True,False,False,False,True,False,True,False,False,False
1,35.60,247.25,06754X0077/F1,5.356637,46.028102,2020-01-01,2.0,196.0,N,5.329333,...,True,False,False,False,True,False,True,False,False,False
2,35.22,218.77,06993X0226/MEXI_2,5.220795,45.895734,2020-01-01,2.0,196.0,N,5.106667,...,True,False,False,False,True,False,True,False,False,False
3,34.20,499.85,06533X0070/F2,5.948977,46.201180,2020-01-01,2.0,196.0,N,5.843000,...,True,False,False,False,True,False,True,False,False,False
4,37.30,260.00,06518X0026/P2,5.313353,46.136402,2020-01-01,2.0,196.0,N,5.287667,...,True,False,False,False,True,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294080,24.00,63.00,01258X0020/S1,1.711584,49.197517,2023-05-31,2.0,238.0,N,1.895000,...,True,False,False,False,True,False,True,False,False,False
3294081,25.50,103.00,01266X1013/S1,1.901276,49.145695,2023-05-31,2.0,291.0,N,1.895000,...,True,False,False,False,True,False,True,False,False,False
3294082,30.00,83.00,01522X0069/PZ2008,1.897576,49.083846,2023-05-31,2.0,291.0,N,1.895000,...,True,False,False,False,True,False,True,False,False,False


In [34]:
df['piezo_continuity_code'].isna().mean()

np.float64(0.0)

In [35]:
df['piezo_continuity_code'].value_counts()

piezo_continuity_code
2.0    2827480
1.0       2592
Name: count, dtype: int64

In [36]:
one_hot_dummies = pd.get_dummies(df['piezo_continuity_code'], prefix='piezo_continuity_code')
# Combining the one-hot encoded columns with the original DataFrame
df_cleaned = pd.concat([df_cleaned.drop('piezo_continuity_code', axis=1), one_hot_dummies], axis=1)
df_cleaned

,piezo_station_investigation_depth,piezo_station_altitude,piezo_station_bss_code,piezo_station_longitude,piezo_station_latitude,piezo_measurement_date,piezo_producer_code,piezo_measure_nature_code,meteo_latitude,meteo_longitude,...,piezo_status_Donnée brute,piezo_status_Donnée contrôlée niveau 1,piezo_status_Donnée contrôlée niveau 2,piezo_status_Donnée interprétée,piezo_qualification_Correcte,piezo_qualification_Incertaine,piezo_qualification_Incorrecte,piezo_qualification_Non qualifié,piezo_continuity_code_1.0,piezo_continuity_code_2.0
row_index,,,,,,,,,,,,,,,,,,,,,
0,20.00,232.00,07004X0046/D6-20,5.748241,45.828148,2020-01-01,196.0,N,5.688000,45.769333,...,False,False,True,False,True,False,False,False,False,True
1,35.60,247.25,06754X0077/F1,5.356637,46.028102,2020-01-01,196.0,N,5.329333,45.976500,...,False,False,True,False,True,False,False,False,False,True
2,35.22,218.77,06993X0226/MEXI_2,5.220795,45.895734,2020-01-01,196.0,N,5.106667,45.833000,...,False,False,True,False,True,False,False,False,False,True
3,34.20,499.85,06533X0070/F2,5.948977,46.201180,2020-01-01,196.0,N,5.843000,46.302833,...,False,False,True,False,True,False,False,False,False,True
4,37.30,260.00,06518X0026/P2,5.313353,46.136402,2020-01-01,196.0,N,5.287667,46.204333,...,False,False,True,False,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294080,24.00,63.00,01258X0020/S1,1.711584,49.197517,2023-05-31,238.0,N,1.895000,49.193333,...,False,False,True,False,True,False,False,False,False,True
3294081,25.50,103.00,01266X1013/S1,1.901276,49.145695,2023-05-31,291.0,N,1.895000,49.193333,...,False,False,True,False,True,False,False,False,False,True
3294082,30.00,83.00,01522X0069/PZ2008,1.897576,49.083846,2023-05-31,291.0,N,1.895000,49.193333,...,False,False,True,False,True,False,False,False,False,True


In [37]:
df['piezo_producer_code'].isna().mean()

np.float64(0.05152589757433733)

In [38]:
df['piezo_producer_code'].value_counts()

piezo_producer_code
3.270000e+02    300284
2.650000e+02    198730
2.930000e+02    194185
1.219000e+03    169223
2.910000e+02    163531
1.960000e+02    118662
4.049439e+13    112892
2.940000e+02    112497
2.386000e+13     98660
2.240000e+13     98547
1.980000e+02     94789
2.960000e+02     91359
3.230000e+02     75906
3.000000e+02     74846
1.970000e+02     60151
2.380000e+02     60003
1.218000e+03     58219
2.970000e+02     57843
8.650000e+02     54810
2.000000e+02     46589
2.285000e+13     46311
2.517018e+13     42320
2.310000e+02     31438
2.950000e+02     29880
2.000158e+13     28678
2.000021e+13     23956
1.990000e+02     23159
1.296000e+03     21201
1.299000e+03     20886
2.272000e+13     17875
2.206000e+13     17254
2.530019e+13     17198
1.615000e+03     14498
2.211000e+13     14214
1.275000e+03     12741
2.284000e+13     10467
8.420000e+02      9162
2.209000e+13      8957
2.534035e+13      8871
2.530027e+13      8151
2.505013e+13      6750
2.990000e+02      6009
8.560000e+02  

In [39]:
one_hot_dummies = pd.get_dummies(df['piezo_producer_code'], prefix='piezo_producer_code')
# Combining the one-hot encoded columns with the original DataFrame
df_cleaned = pd.concat([df_cleaned.drop('piezo_producer_code', axis=1), one_hot_dummies], axis=1)
df_cleaned

,piezo_station_investigation_depth,piezo_station_altitude,piezo_station_bss_code,piezo_station_longitude,piezo_station_latitude,piezo_measurement_date,piezo_measure_nature_code,meteo_latitude,meteo_longitude,meteo_altitude,...,piezo_producer_code_22850001300658.0,piezo_producer_code_23860001900042.0,piezo_producer_code_25050128500028.0,piezo_producer_code_25170181900012.0,piezo_producer_code_25270240200014.0,piezo_producer_code_25300186100017.0,piezo_producer_code_25300271100021.0,piezo_producer_code_25340353900039.0,piezo_producer_code_40494388800036.0,piezo_producer_code_49302112500031.0
row_index,,,,,,,,,,,,,,,,,,,,,
0,20.00,232.00,07004X0046/D6-20,5.748241,45.828148,2020-01-01,N,5.688000,45.769333,330,...,False,False,False,False,False,False,False,False,False,False
1,35.60,247.25,06754X0077/F1,5.356637,46.028102,2020-01-01,N,5.329333,45.976500,250,...,False,False,False,False,False,False,False,False,False,False
2,35.22,218.77,06993X0226/MEXI_2,5.220795,45.895734,2020-01-01,N,5.106667,45.833000,196,...,False,False,False,False,False,False,False,False,False,False
3,34.20,499.85,06533X0070/F2,5.948977,46.201180,2020-01-01,N,5.843000,46.302833,1133,...,False,False,False,False,False,False,False,False,False,False
4,37.30,260.00,06518X0026/P2,5.313353,46.136402,2020-01-01,N,5.287667,46.204333,260,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294080,24.00,63.00,01258X0020/S1,1.711584,49.197517,2023-05-31,N,1.895000,49.193333,105,...,False,False,False,False,False,False,False,False,False,False
3294081,25.50,103.00,01266X1013/S1,1.901276,49.145695,2023-05-31,N,1.895000,49.193333,105,...,False,False,False,False,False,False,False,False,False,False
3294082,30.00,83.00,01522X0069/PZ2008,1.897576,49.083846,2023-05-31,N,1.895000,49.193333,105,...,False,False,False,False,False,False,False,False,False,False


In [40]:
df['piezo_measure_nature_code'].isna().mean()

np.float64(0.0397113571668848)

In [41]:
df['piezo_measure_nature_code'].value_counts()

piezo_measure_nature_code
0    1422458
N    1151930
I     121262
D      19583
S       2453
Name: count, dtype: int64

In [42]:
one_hot_dummies = pd.get_dummies(df['piezo_measure_nature_code'], prefix='piezo_measure_nature_code')
# Combining the one-hot encoded columns with the original DataFrame
df_cleaned = pd.concat([df_cleaned.drop('piezo_measure_nature_code', axis=1), one_hot_dummies], axis=1)
df_cleaned

,piezo_station_investigation_depth,piezo_station_altitude,piezo_station_bss_code,piezo_station_longitude,piezo_station_latitude,piezo_measurement_date,meteo_latitude,meteo_longitude,meteo_altitude,meteo_rain_height,...,piezo_producer_code_25300186100017.0,piezo_producer_code_25300271100021.0,piezo_producer_code_25340353900039.0,piezo_producer_code_40494388800036.0,piezo_producer_code_49302112500031.0,piezo_measure_nature_code_0,piezo_measure_nature_code_D,piezo_measure_nature_code_I,piezo_measure_nature_code_N,piezo_measure_nature_code_S
row_index,,,,,,,,,,,,,,,,,,,,,
0,20.00,232.00,07004X0046/D6-20,5.748241,45.828148,2020-01-01,5.688000,45.769333,330,0.2,...,False,False,False,False,False,False,False,False,True,False
1,35.60,247.25,06754X0077/F1,5.356637,46.028102,2020-01-01,5.329333,45.976500,250,0.0,...,False,False,False,False,False,False,False,False,True,False
2,35.22,218.77,06993X0226/MEXI_2,5.220795,45.895734,2020-01-01,5.106667,45.833000,196,NaN,...,False,False,False,False,False,False,False,False,True,False
3,34.20,499.85,06533X0070/F2,5.948977,46.201180,2020-01-01,5.843000,46.302833,1133,0.0,...,False,False,False,False,False,False,False,False,True,False
4,37.30,260.00,06518X0026/P2,5.313353,46.136402,2020-01-01,5.287667,46.204333,260,0.0,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294080,24.00,63.00,01258X0020/S1,1.711584,49.197517,2023-05-31,1.895000,49.193333,105,0.0,...,False,False,False,False,False,False,False,False,True,False
3294081,25.50,103.00,01266X1013/S1,1.901276,49.145695,2023-05-31,1.895000,49.193333,105,0.0,...,False,False,False,False,False,False,False,False,True,False
3294082,30.00,83.00,01522X0069/PZ2008,1.897576,49.083846,2023-05-31,1.895000,49.193333,105,0.0,...,False,False,False,False,False,False,False,False,True,False


In [43]:
df_cleaned.memory_usage().sum() / 1e9

np.float64(2.864032864)

In [44]:
df_cleaned['piezo_groundwater_level_category']

row_index
0               High
1          Very High
2               High
3          Very High
4           Very Low
             ...    
3294080     Very Low
3294081     Very Low
3294082          Low
3294083         High
3294084      Average
Name: piezo_groundwater_level_category, Length: 2830072, dtype: object

## Météo

In [45]:
for column in df.columns:
    if column[:5] == 'meteo':
        if df.dtypes[column] != 'float64':
            print(column, df.dtypes[column])

meteo_altitude int64


In [46]:
df_cleaned.memory_usage().sum() / 1e9

np.float64(2.864032864)

## Hydrologie

In [47]:
for column in df.columns:
    if column[:5] == 'hydro':
        if df.dtypes[column] != 'float64':
            print(column, df.dtypes[column])

hydro_status_code int64
hydro_qualification_code int64
hydro_hydro_quantity_elab object


In [48]:
df['hydro_status_code'].isna().mean()

np.float64(0.0)

In [49]:
df['hydro_status_code'].value_counts()

hydro_status_code
16    1480308
12    1189889
4      148852
8       11023
Name: count, dtype: int64

In [50]:
one_hot_dummies = pd.get_dummies(df['hydro_status_code'], prefix='hydro_status_code')
# Combining the one-hot encoded columns with the original DataFrame
df_cleaned = pd.concat([df_cleaned.drop('hydro_status_code', axis=1), one_hot_dummies], axis=1)
df_cleaned

,piezo_station_investigation_depth,piezo_station_altitude,piezo_station_bss_code,piezo_station_longitude,piezo_station_latitude,piezo_measurement_date,meteo_latitude,meteo_longitude,meteo_altitude,meteo_rain_height,...,piezo_producer_code_49302112500031.0,piezo_measure_nature_code_0,piezo_measure_nature_code_D,piezo_measure_nature_code_I,piezo_measure_nature_code_N,piezo_measure_nature_code_S,hydro_status_code_4,hydro_status_code_8,hydro_status_code_12,hydro_status_code_16
row_index,,,,,,,,,,,,,,,,,,,,,
0,20.00,232.00,07004X0046/D6-20,5.748241,45.828148,2020-01-01,5.688000,45.769333,330,0.2,...,False,False,False,False,True,False,False,False,False,True
1,35.60,247.25,06754X0077/F1,5.356637,46.028102,2020-01-01,5.329333,45.976500,250,0.0,...,False,False,False,False,True,False,False,False,False,True
2,35.22,218.77,06993X0226/MEXI_2,5.220795,45.895734,2020-01-01,5.106667,45.833000,196,NaN,...,False,False,False,False,True,False,False,False,False,True
3,34.20,499.85,06533X0070/F2,5.948977,46.201180,2020-01-01,5.843000,46.302833,1133,0.0,...,False,False,False,False,True,False,False,False,False,True
4,37.30,260.00,06518X0026/P2,5.313353,46.136402,2020-01-01,5.287667,46.204333,260,0.0,...,False,False,False,False,True,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294080,24.00,63.00,01258X0020/S1,1.711584,49.197517,2023-05-31,1.895000,49.193333,105,0.0,...,False,False,False,False,True,False,False,False,True,False
3294081,25.50,103.00,01266X1013/S1,1.901276,49.145695,2023-05-31,1.895000,49.193333,105,0.0,...,False,False,False,False,True,False,False,False,False,True
3294082,30.00,83.00,01522X0069/PZ2008,1.897576,49.083846,2023-05-31,1.895000,49.193333,105,0.0,...,False,False,False,False,True,False,False,False,False,True


In [51]:
df['hydro_qualification_code'].isna().mean()

np.float64(0.0)

In [52]:
df['hydro_qualification_code'].value_counts()

hydro_qualification_code
20    2036744
12     422346
16     370982
Name: count, dtype: int64

In [53]:
one_hot_dummies = pd.get_dummies(df['hydro_qualification_code'], prefix='hydro_qualification_code')
# Combining the one-hot encoded columns with the original DataFrame
df_cleaned = pd.concat([df_cleaned.drop('hydro_qualification_code', axis=1), one_hot_dummies], axis=1)
df_cleaned

,piezo_station_investigation_depth,piezo_station_altitude,piezo_station_bss_code,piezo_station_longitude,piezo_station_latitude,piezo_measurement_date,meteo_latitude,meteo_longitude,meteo_altitude,meteo_rain_height,...,piezo_measure_nature_code_I,piezo_measure_nature_code_N,piezo_measure_nature_code_S,hydro_status_code_4,hydro_status_code_8,hydro_status_code_12,hydro_status_code_16,hydro_qualification_code_12,hydro_qualification_code_16,hydro_qualification_code_20
row_index,,,,,,,,,,,,,,,,,,,,,
0,20.00,232.00,07004X0046/D6-20,5.748241,45.828148,2020-01-01,5.688000,45.769333,330,0.2,...,False,True,False,False,False,False,True,False,False,True
1,35.60,247.25,06754X0077/F1,5.356637,46.028102,2020-01-01,5.329333,45.976500,250,0.0,...,False,True,False,False,False,False,True,False,False,True
2,35.22,218.77,06993X0226/MEXI_2,5.220795,45.895734,2020-01-01,5.106667,45.833000,196,NaN,...,False,True,False,False,False,False,True,False,False,True
3,34.20,499.85,06533X0070/F2,5.948977,46.201180,2020-01-01,5.843000,46.302833,1133,0.0,...,False,True,False,False,False,False,True,True,False,False
4,37.30,260.00,06518X0026/P2,5.313353,46.136402,2020-01-01,5.287667,46.204333,260,0.0,...,False,True,False,False,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294080,24.00,63.00,01258X0020/S1,1.711584,49.197517,2023-05-31,1.895000,49.193333,105,0.0,...,False,True,False,False,False,True,False,False,False,True
3294081,25.50,103.00,01266X1013/S1,1.901276,49.145695,2023-05-31,1.895000,49.193333,105,0.0,...,False,True,False,False,False,False,True,False,True,False
3294082,30.00,83.00,01522X0069/PZ2008,1.897576,49.083846,2023-05-31,1.895000,49.193333,105,0.0,...,False,True,False,False,False,False,True,False,True,False


In [54]:
df['hydro_hydro_quantity_elab'].isna().mean()

np.float64(0.0)

In [55]:
df['hydro_hydro_quantity_elab'].value_counts()

hydro_hydro_quantity_elab
QmJ    2784583
QmM      45489
Name: count, dtype: int64

In [56]:
one_hot_dummies = pd.get_dummies(df['hydro_hydro_quantity_elab'], prefix='hydro_hydro_quantity_elab')
# Combining the one-hot encoded columns with the original DataFrame
df_cleaned = pd.concat([df_cleaned.drop('hydro_hydro_quantity_elab', axis=1), one_hot_dummies], axis=1)
df_cleaned

,piezo_station_investigation_depth,piezo_station_altitude,piezo_station_bss_code,piezo_station_longitude,piezo_station_latitude,piezo_measurement_date,meteo_latitude,meteo_longitude,meteo_altitude,meteo_rain_height,...,piezo_measure_nature_code_S,hydro_status_code_4,hydro_status_code_8,hydro_status_code_12,hydro_status_code_16,hydro_qualification_code_12,hydro_qualification_code_16,hydro_qualification_code_20,hydro_hydro_quantity_elab_QmJ,hydro_hydro_quantity_elab_QmM
row_index,,,,,,,,,,,,,,,,,,,,,
0,20.00,232.00,07004X0046/D6-20,5.748241,45.828148,2020-01-01,5.688000,45.769333,330,0.2,...,False,False,False,False,True,False,False,True,True,False
1,35.60,247.25,06754X0077/F1,5.356637,46.028102,2020-01-01,5.329333,45.976500,250,0.0,...,False,False,False,False,True,False,False,True,False,True
2,35.22,218.77,06993X0226/MEXI_2,5.220795,45.895734,2020-01-01,5.106667,45.833000,196,NaN,...,False,False,False,False,True,False,False,True,True,False
3,34.20,499.85,06533X0070/F2,5.948977,46.201180,2020-01-01,5.843000,46.302833,1133,0.0,...,False,False,False,False,True,True,False,False,False,True
4,37.30,260.00,06518X0026/P2,5.313353,46.136402,2020-01-01,5.287667,46.204333,260,0.0,...,False,False,False,True,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294080,24.00,63.00,01258X0020/S1,1.711584,49.197517,2023-05-31,1.895000,49.193333,105,0.0,...,False,False,False,True,False,False,False,True,True,False
3294081,25.50,103.00,01266X1013/S1,1.901276,49.145695,2023-05-31,1.895000,49.193333,105,0.0,...,False,False,False,False,True,False,True,False,True,False
3294082,30.00,83.00,01522X0069/PZ2008,1.897576,49.083846,2023-05-31,1.895000,49.193333,105,0.0,...,False,False,False,False,True,False,True,False,True,False


## Prélèvement

In [57]:
for column in df.columns:
    if column[:5] == 'prele':
        if df.dtypes[column] != 'float64':
            print(column, df.dtypes[column])

prelev_usage_label_0 object
prelev_volume_obtention_mode_label_0 object
prelev_usage_label_1 object
prelev_volume_obtention_mode_label_1 object
prelev_usage_label_2 object
prelev_volume_obtention_mode_label_2 object


In [58]:
df['prelev_usage_label_0'].isna().mean()

np.float64(0.19530527845227966)

In [59]:
df['prelev_usage_label_0'].value_counts()

prelev_usage_label_0
EAU POTABLE                                                           974051
EAU TURBINEE (barrage)                                                572668
INDUSTRIE et ACTIVITES ECONOMIQUES (hors irrigation, hors énergie)    247423
CANAUX                                                                218769
IRRIGATION                                                            175327
ENERGIE                                                                89106
Name: count, dtype: int64

In [60]:
one_hot_dummies = pd.get_dummies(df['prelev_usage_label_0'], prefix='prelev_usage_label_0')
# Combining the one-hot encoded columns with the original DataFrame
df_cleaned = pd.concat([df_cleaned.drop('prelev_usage_label_0', axis=1), one_hot_dummies], axis=1)
df_cleaned

,piezo_station_investigation_depth,piezo_station_altitude,piezo_station_bss_code,piezo_station_longitude,piezo_station_latitude,piezo_measurement_date,meteo_latitude,meteo_longitude,meteo_altitude,meteo_rain_height,...,hydro_qualification_code_16,hydro_qualification_code_20,hydro_hydro_quantity_elab_QmJ,hydro_hydro_quantity_elab_QmM,prelev_usage_label_0_CANAUX,prelev_usage_label_0_EAU POTABLE,prelev_usage_label_0_EAU TURBINEE (barrage),prelev_usage_label_0_ENERGIE,"prelev_usage_label_0_INDUSTRIE et ACTIVITES ECONOMIQUES (hors irrigation, hors énergie)",prelev_usage_label_0_IRRIGATION
row_index,,,,,,,,,,,,,,,,,,,,,
0,20.00,232.00,07004X0046/D6-20,5.748241,45.828148,2020-01-01,5.688000,45.769333,330,0.2,...,False,True,True,False,False,False,True,False,False,False
1,35.60,247.25,06754X0077/F1,5.356637,46.028102,2020-01-01,5.329333,45.976500,250,0.0,...,False,True,False,True,False,False,True,False,False,False
2,35.22,218.77,06993X0226/MEXI_2,5.220795,45.895734,2020-01-01,5.106667,45.833000,196,NaN,...,False,True,True,False,False,False,True,False,False,False
3,34.20,499.85,06533X0070/F2,5.948977,46.201180,2020-01-01,5.843000,46.302833,1133,0.0,...,False,False,False,True,False,False,True,False,False,False
4,37.30,260.00,06518X0026/P2,5.313353,46.136402,2020-01-01,5.287667,46.204333,260,0.0,...,False,True,True,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294080,24.00,63.00,01258X0020/S1,1.711584,49.197517,2023-05-31,1.895000,49.193333,105,0.0,...,False,True,True,False,False,False,False,False,False,False
3294081,25.50,103.00,01266X1013/S1,1.901276,49.145695,2023-05-31,1.895000,49.193333,105,0.0,...,True,False,True,False,False,False,False,False,False,False
3294082,30.00,83.00,01522X0069/PZ2008,1.897576,49.083846,2023-05-31,1.895000,49.193333,105,0.0,...,True,False,True,False,False,False,False,False,False,False


In [61]:
df['prelev_volume_obtention_mode_label_0'].isna().mean()

np.float64(0.19530527845227966)

In [62]:
df['prelev_volume_obtention_mode_label_0'].value_counts()

prelev_volume_obtention_mode_label_0
Mesure directe        1173258
Volume mesuré          397486
Mesure indirecte       319650
Volume estimé          295868
Volume forfaitaire      91082
Name: count, dtype: int64

In [63]:
one_hot_dummies = pd.get_dummies(df['prelev_volume_obtention_mode_label_0'], prefix='prelev_volume_obtention_mode_label_0')
# Combining the one-hot encoded columns with the original DataFrame
df_cleaned = pd.concat([df_cleaned.drop('prelev_volume_obtention_mode_label_0', axis=1), one_hot_dummies], axis=1)
df_cleaned

,piezo_station_investigation_depth,piezo_station_altitude,piezo_station_bss_code,piezo_station_longitude,piezo_station_latitude,piezo_measurement_date,meteo_latitude,meteo_longitude,meteo_altitude,meteo_rain_height,...,prelev_usage_label_0_EAU POTABLE,prelev_usage_label_0_EAU TURBINEE (barrage),prelev_usage_label_0_ENERGIE,"prelev_usage_label_0_INDUSTRIE et ACTIVITES ECONOMIQUES (hors irrigation, hors énergie)",prelev_usage_label_0_IRRIGATION,prelev_volume_obtention_mode_label_0_Mesure directe,prelev_volume_obtention_mode_label_0_Mesure indirecte,prelev_volume_obtention_mode_label_0_Volume estimé,prelev_volume_obtention_mode_label_0_Volume forfaitaire,prelev_volume_obtention_mode_label_0_Volume mesuré
row_index,,,,,,,,,,,,,,,,,,,,,
0,20.00,232.00,07004X0046/D6-20,5.748241,45.828148,2020-01-01,5.688000,45.769333,330,0.2,...,False,True,False,False,False,False,True,False,False,False
1,35.60,247.25,06754X0077/F1,5.356637,46.028102,2020-01-01,5.329333,45.976500,250,0.0,...,False,True,False,False,False,False,True,False,False,False
2,35.22,218.77,06993X0226/MEXI_2,5.220795,45.895734,2020-01-01,5.106667,45.833000,196,NaN,...,False,True,False,False,False,False,True,False,False,False
3,34.20,499.85,06533X0070/F2,5.948977,46.201180,2020-01-01,5.843000,46.302833,1133,0.0,...,False,True,False,False,False,False,True,False,False,False
4,37.30,260.00,06518X0026/P2,5.313353,46.136402,2020-01-01,5.287667,46.204333,260,0.0,...,False,True,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294080,24.00,63.00,01258X0020/S1,1.711584,49.197517,2023-05-31,1.895000,49.193333,105,0.0,...,False,False,False,False,False,False,False,False,False,False
3294081,25.50,103.00,01266X1013/S1,1.901276,49.145695,2023-05-31,1.895000,49.193333,105,0.0,...,False,False,False,False,False,False,False,False,False,False
3294082,30.00,83.00,01522X0069/PZ2008,1.897576,49.083846,2023-05-31,1.895000,49.193333,105,0.0,...,False,False,False,False,False,False,False,False,False,False


In [64]:
df['prelev_usage_label_1'].isna().mean()

np.float64(0.19556286907188228)

In [65]:
df['prelev_usage_label_1'].value_counts()

prelev_usage_label_1
EAU POTABLE                                                           1158258
EAU TURBINEE (barrage)                                                 402367
IRRIGATION                                                             269847
INDUSTRIE et ACTIVITES ECONOMIQUES (hors irrigation, hors énergie)     253556
CANAUX                                                                 143806
ENERGIE                                                                 48781
Name: count, dtype: int64

In [66]:
one_hot_dummies = pd.get_dummies(df['prelev_usage_label_1'], prefix='prelev_usage_label_1')
# Combining the one-hot encoded columns with the original DataFrame
df_cleaned = pd.concat([df_cleaned.drop('prelev_usage_label_1', axis=1), one_hot_dummies], axis=1)
df_cleaned

,piezo_station_investigation_depth,piezo_station_altitude,piezo_station_bss_code,piezo_station_longitude,piezo_station_latitude,piezo_measurement_date,meteo_latitude,meteo_longitude,meteo_altitude,meteo_rain_height,...,prelev_volume_obtention_mode_label_0_Mesure indirecte,prelev_volume_obtention_mode_label_0_Volume estimé,prelev_volume_obtention_mode_label_0_Volume forfaitaire,prelev_volume_obtention_mode_label_0_Volume mesuré,prelev_usage_label_1_CANAUX,prelev_usage_label_1_EAU POTABLE,prelev_usage_label_1_EAU TURBINEE (barrage),prelev_usage_label_1_ENERGIE,"prelev_usage_label_1_INDUSTRIE et ACTIVITES ECONOMIQUES (hors irrigation, hors énergie)",prelev_usage_label_1_IRRIGATION
row_index,,,,,,,,,,,,,,,,,,,,,
0,20.00,232.00,07004X0046/D6-20,5.748241,45.828148,2020-01-01,5.688000,45.769333,330,0.2,...,True,False,False,False,False,False,True,False,False,False
1,35.60,247.25,06754X0077/F1,5.356637,46.028102,2020-01-01,5.329333,45.976500,250,0.0,...,True,False,False,False,False,False,True,False,False,False
2,35.22,218.77,06993X0226/MEXI_2,5.220795,45.895734,2020-01-01,5.106667,45.833000,196,NaN,...,True,False,False,False,False,False,False,True,False,False
3,34.20,499.85,06533X0070/F2,5.948977,46.201180,2020-01-01,5.843000,46.302833,1133,0.0,...,True,False,False,False,False,False,True,False,False,False
4,37.30,260.00,06518X0026/P2,5.313353,46.136402,2020-01-01,5.287667,46.204333,260,0.0,...,True,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294080,24.00,63.00,01258X0020/S1,1.711584,49.197517,2023-05-31,1.895000,49.193333,105,0.0,...,False,False,False,False,False,False,False,False,False,False
3294081,25.50,103.00,01266X1013/S1,1.901276,49.145695,2023-05-31,1.895000,49.193333,105,0.0,...,False,False,False,False,False,False,False,False,False,False
3294082,30.00,83.00,01522X0069/PZ2008,1.897576,49.083846,2023-05-31,1.895000,49.193333,105,0.0,...,False,False,False,False,False,False,False,False,False,False


In [67]:
df['prelev_volume_obtention_mode_label_1'].isna().mean()

np.float64(0.19556286907188228)

In [68]:
df['prelev_volume_obtention_mode_label_1'].value_counts()

prelev_volume_obtention_mode_label_1
Mesure directe        1257667
Volume mesuré          463224
Mesure indirecte       286292
Volume estimé          170595
Volume forfaitaire      98837
Name: count, dtype: int64

In [69]:
one_hot_dummies = pd.get_dummies(df['prelev_volume_obtention_mode_label_1'], prefix='prelev_volume_obtention_mode_label_1')
# Combining the one-hot encoded columns with the original DataFrame
df_cleaned = pd.concat([df_cleaned.drop('prelev_volume_obtention_mode_label_1', axis=1), one_hot_dummies], axis=1)
df_cleaned

,piezo_station_investigation_depth,piezo_station_altitude,piezo_station_bss_code,piezo_station_longitude,piezo_station_latitude,piezo_measurement_date,meteo_latitude,meteo_longitude,meteo_altitude,meteo_rain_height,...,prelev_usage_label_1_EAU POTABLE,prelev_usage_label_1_EAU TURBINEE (barrage),prelev_usage_label_1_ENERGIE,"prelev_usage_label_1_INDUSTRIE et ACTIVITES ECONOMIQUES (hors irrigation, hors énergie)",prelev_usage_label_1_IRRIGATION,prelev_volume_obtention_mode_label_1_Mesure directe,prelev_volume_obtention_mode_label_1_Mesure indirecte,prelev_volume_obtention_mode_label_1_Volume estimé,prelev_volume_obtention_mode_label_1_Volume forfaitaire,prelev_volume_obtention_mode_label_1_Volume mesuré
row_index,,,,,,,,,,,,,,,,,,,,,
0,20.00,232.00,07004X0046/D6-20,5.748241,45.828148,2020-01-01,5.688000,45.769333,330,0.2,...,False,True,False,False,False,False,True,False,False,False
1,35.60,247.25,06754X0077/F1,5.356637,46.028102,2020-01-01,5.329333,45.976500,250,0.0,...,False,True,False,False,False,False,True,False,False,False
2,35.22,218.77,06993X0226/MEXI_2,5.220795,45.895734,2020-01-01,5.106667,45.833000,196,NaN,...,False,False,True,False,False,False,True,False,False,False
3,34.20,499.85,06533X0070/F2,5.948977,46.201180,2020-01-01,5.843000,46.302833,1133,0.0,...,False,True,False,False,False,False,True,False,False,False
4,37.30,260.00,06518X0026/P2,5.313353,46.136402,2020-01-01,5.287667,46.204333,260,0.0,...,False,True,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294080,24.00,63.00,01258X0020/S1,1.711584,49.197517,2023-05-31,1.895000,49.193333,105,0.0,...,False,False,False,False,False,False,False,False,False,False
3294081,25.50,103.00,01266X1013/S1,1.901276,49.145695,2023-05-31,1.895000,49.193333,105,0.0,...,False,False,False,False,False,False,False,False,False,False
3294082,30.00,83.00,01522X0069/PZ2008,1.897576,49.083846,2023-05-31,1.895000,49.193333,105,0.0,...,False,False,False,False,False,False,False,False,False,False


In [70]:
df['prelev_usage_label_2'].isna().mean()

np.float64(0.19642221116635902)

In [71]:
df['prelev_usage_label_2'].value_counts()

prelev_usage_label_2
EAU POTABLE                                                           1241401
EAU TURBINEE (barrage)                                                 312891
IRRIGATION                                                             282025
INDUSTRIE et ACTIVITES ECONOMIQUES (hors irrigation, hors énergie)     258495
CANAUX                                                                 135164
ENERGIE                                                                 44207
Name: count, dtype: int64

In [72]:
one_hot_dummies = pd.get_dummies(df['prelev_usage_label_2'], prefix='prelev_usage_label_2')
# Combining the one-hot encoded columns with the original DataFrame
df_cleaned = pd.concat([df_cleaned.drop('prelev_usage_label_2', axis=1), one_hot_dummies], axis=1)
df_cleaned

,piezo_station_investigation_depth,piezo_station_altitude,piezo_station_bss_code,piezo_station_longitude,piezo_station_latitude,piezo_measurement_date,meteo_latitude,meteo_longitude,meteo_altitude,meteo_rain_height,...,prelev_volume_obtention_mode_label_1_Mesure indirecte,prelev_volume_obtention_mode_label_1_Volume estimé,prelev_volume_obtention_mode_label_1_Volume forfaitaire,prelev_volume_obtention_mode_label_1_Volume mesuré,prelev_usage_label_2_CANAUX,prelev_usage_label_2_EAU POTABLE,prelev_usage_label_2_EAU TURBINEE (barrage),prelev_usage_label_2_ENERGIE,"prelev_usage_label_2_INDUSTRIE et ACTIVITES ECONOMIQUES (hors irrigation, hors énergie)",prelev_usage_label_2_IRRIGATION
row_index,,,,,,,,,,,,,,,,,,,,,
0,20.00,232.00,07004X0046/D6-20,5.748241,45.828148,2020-01-01,5.688000,45.769333,330,0.2,...,True,False,False,False,False,False,True,False,False,False
1,35.60,247.25,06754X0077/F1,5.356637,46.028102,2020-01-01,5.329333,45.976500,250,0.0,...,True,False,False,False,False,False,True,False,False,False
2,35.22,218.77,06993X0226/MEXI_2,5.220795,45.895734,2020-01-01,5.106667,45.833000,196,NaN,...,True,False,False,False,False,False,True,False,False,False
3,34.20,499.85,06533X0070/F2,5.948977,46.201180,2020-01-01,5.843000,46.302833,1133,0.0,...,True,False,False,False,False,False,True,False,False,False
4,37.30,260.00,06518X0026/P2,5.313353,46.136402,2020-01-01,5.287667,46.204333,260,0.0,...,True,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294080,24.00,63.00,01258X0020/S1,1.711584,49.197517,2023-05-31,1.895000,49.193333,105,0.0,...,False,False,False,False,False,False,False,False,False,False
3294081,25.50,103.00,01266X1013/S1,1.901276,49.145695,2023-05-31,1.895000,49.193333,105,0.0,...,False,False,False,False,False,False,False,False,False,False
3294082,30.00,83.00,01522X0069/PZ2008,1.897576,49.083846,2023-05-31,1.895000,49.193333,105,0.0,...,False,False,False,False,False,False,False,False,False,False


In [73]:
df['prelev_volume_obtention_mode_label_2'].isna().mean()

np.float64(0.19642221116635902)

In [74]:
df['prelev_volume_obtention_mode_label_2'].value_counts()

prelev_volume_obtention_mode_label_2
Mesure directe        1335887
Volume mesuré          491528
Mesure indirecte       251718
Volume estimé          127866
Volume forfaitaire      67184
Name: count, dtype: int64

In [75]:
one_hot_dummies = pd.get_dummies(df['prelev_volume_obtention_mode_label_2'], prefix='prelev_volume_obtention_mode_label_2')
# Combining the one-hot encoded columns with the original DataFrame
df_cleaned = pd.concat([df_cleaned.drop('prelev_volume_obtention_mode_label_2', axis=1), one_hot_dummies], axis=1)
df_cleaned

,piezo_station_investigation_depth,piezo_station_altitude,piezo_station_bss_code,piezo_station_longitude,piezo_station_latitude,piezo_measurement_date,meteo_latitude,meteo_longitude,meteo_altitude,meteo_rain_height,...,prelev_usage_label_2_EAU POTABLE,prelev_usage_label_2_EAU TURBINEE (barrage),prelev_usage_label_2_ENERGIE,"prelev_usage_label_2_INDUSTRIE et ACTIVITES ECONOMIQUES (hors irrigation, hors énergie)",prelev_usage_label_2_IRRIGATION,prelev_volume_obtention_mode_label_2_Mesure directe,prelev_volume_obtention_mode_label_2_Mesure indirecte,prelev_volume_obtention_mode_label_2_Volume estimé,prelev_volume_obtention_mode_label_2_Volume forfaitaire,prelev_volume_obtention_mode_label_2_Volume mesuré
row_index,,,,,,,,,,,,,,,,,,,,,
0,20.00,232.00,07004X0046/D6-20,5.748241,45.828148,2020-01-01,5.688000,45.769333,330,0.2,...,False,True,False,False,False,False,True,False,False,False
1,35.60,247.25,06754X0077/F1,5.356637,46.028102,2020-01-01,5.329333,45.976500,250,0.0,...,False,True,False,False,False,False,True,False,False,False
2,35.22,218.77,06993X0226/MEXI_2,5.220795,45.895734,2020-01-01,5.106667,45.833000,196,NaN,...,False,True,False,False,False,False,True,False,False,False
3,34.20,499.85,06533X0070/F2,5.948977,46.201180,2020-01-01,5.843000,46.302833,1133,0.0,...,False,True,False,False,False,False,True,False,False,False
4,37.30,260.00,06518X0026/P2,5.313353,46.136402,2020-01-01,5.287667,46.204333,260,0.0,...,False,True,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294080,24.00,63.00,01258X0020/S1,1.711584,49.197517,2023-05-31,1.895000,49.193333,105,0.0,...,False,False,False,False,False,False,False,False,False,False
3294081,25.50,103.00,01266X1013/S1,1.901276,49.145695,2023-05-31,1.895000,49.193333,105,0.0,...,False,False,False,False,False,False,False,False,False,False
3294082,30.00,83.00,01522X0069/PZ2008,1.897576,49.083846,2023-05-31,1.895000,49.193333,105,0.0,...,False,False,False,False,False,False,False,False,False,False


In [76]:
df_cleaned.memory_usage().sum() / 1e9

np.float64(2.779130704)

## BSS_Code

In [77]:
df['piezo_station_bss_code'].isna().mean()

np.float64(0.0)

In [78]:
one_hot_dummies = pd.get_dummies(df['piezo_station_bss_code'], prefix='piezo_station_bss_code')

In [94]:
stations_embeddings = np.random.normal(size=(len(df['piezo_station_bss_code'].unique()), 10))

In [96]:
bss_codes = list(df_cleaned['piezo_station_bss_code'].unique())

In [99]:
bss_code_to_idx = {bss:idx for idx, bss in enumerate(bss_codes)} 
bss_code_to_idx

{'07004X0046/D6-20': 0,
 '06754X0077/F1': 1,
 '06993X0226/MEXI_2': 2,
 '06533X0070/F2': 3,
 '06518X0026/P2': 4,
 '07004X0019/S3': 5,
 '06993X0087/F6': 6,
 '06288X0096/SB': 7,
 '06775X0010/BOURSI': 8,
 '06742X0001/VILLEN': 9,
 'BSS003EFKG/X': 10,
 'BSS003EFOW/X': 11,
 '06512X0037/STREMY': 12,
 '06991X0220/PZ': 13,
 '06757X0071/PZ': 14,
 '06744X0025/PZ': 15,
 '06261X0007/PZ': 16,
 '06503X0028/PZ': 17,
 '06256X0188/PZ': 18,
 '00855X0001/S1': 19,
 '01568X0101/S1': 20,
 '01293X0071/S1': 21,
 '00671X0052/S1': 22,
 '01308X0082/S1': 23,
 '00502X0003/S1': 24,
 '01306X0023/S1': 25,
 '01066X0133/S1': 26,
 '00847X0043/S1': 27,
 '01567X0083/S1': 28,
 '00654X0014/S1': 29,
 '00843X0024/S1': 30,
 '01561X0044/S1': 31,
 '01074X0006/S1': 32,
 '00665X0016/S1': 33,
 '00845X0036/S1': 34,
 '00501X0017/S1': 35,
 '00484X0018/S1': 36,
 '00497X0018/S1': 37,
 '01293X0048/S1': 38,
 '00675X0019/S1': 39,
 '00496X0019/S1': 40,
 '00516X0059/S1': 41,
 '00493X0346/PZ1': 42,
 '01562X0212/PZL2': 43,
 '01562X0208/PZL4': 44

In [100]:
def apply_stations_embeddings(row):
    idx = bss_code_to_idx[row['piezo_station_bss_code']]
    return stations_embeddings[idx, :]

In [101]:
result = df_cleaned.apply(apply_stations_embeddings, axis=1)

# Convert the result to a DataFrame with 10 columns
stations_embeddings = pd.DataFrame(result.tolist(), columns=[f'code_bss_dim_{i}' for i in range(10)])
stations_embeddings

,code_bss_dim_0,code_bss_dim_1,code_bss_dim_2,code_bss_dim_3,code_bss_dim_4,code_bss_dim_5,code_bss_dim_6,code_bss_dim_7,code_bss_dim_8,code_bss_dim_9
0,-0.997383,-1.910414,0.232990,-1.482991,0.049259,-1.805910,0.225231,0.912868,-1.674938,0.318848
1,0.003256,1.029715,-0.219274,0.123016,-1.140928,0.453731,-0.750258,-1.074151,-0.624945,0.425339
2,0.570933,0.702470,-0.011466,0.472359,0.262211,0.080137,-0.609068,2.384162,1.253131,0.151224
3,1.248752,-1.499055,-0.907296,1.905138,-1.290514,1.433205,0.322991,-1.166792,0.120681,1.171185
4,-0.816898,1.060810,-0.166557,0.476222,-0.784821,1.551038,-0.934637,-0.373700,1.738446,-0.279887
...,...,...,...,...,...,...,...,...,...,...
2830067,-0.572225,0.257667,1.564103,1.358191,0.922722,-0.951107,-0.051193,1.563064,-2.186732,-2.003003
2830068,0.052174,0.674796,0.697271,0.472515,1.229737,-1.429845,-2.394821,-0.425498,1.481098,1.726008
2830069,-1.080536,0.172167,-0.653497,0.990581,-1.479457,-0.499743,-0.159442,0.294287,0.113381,-1.150069
2830070,0.689732,-0.055609,-0.624984,-1.397869,-1.312355,-0.428216,1.011750,0.135692,-0.220098,0.545159


In [102]:
# Combining the one-hot encoded columns with the original DataFrame
df_cleaned = pd.concat([df_cleaned, stations_embeddings], axis=1)
df_cleaned

,piezo_station_investigation_depth,piezo_station_altitude,piezo_station_bss_code,piezo_station_longitude,piezo_station_latitude,piezo_measurement_date,meteo_latitude,meteo_longitude,meteo_altitude,meteo_rain_height,...,code_bss_dim_0,code_bss_dim_1,code_bss_dim_2,code_bss_dim_3,code_bss_dim_4,code_bss_dim_5,code_bss_dim_6,code_bss_dim_7,code_bss_dim_8,code_bss_dim_9
0,20.00,232.00,07004X0046/D6-20,5.748241,45.828148,2020-01-01,5.688000,45.769333,330.0,0.2,...,-0.997383,-1.910414,0.232990,-1.482991,0.049259,-1.805910,0.225231,0.912868,-1.674938,0.318848
1,35.60,247.25,06754X0077/F1,5.356637,46.028102,2020-01-01,5.329333,45.976500,250.0,0.0,...,0.003256,1.029715,-0.219274,0.123016,-1.140928,0.453731,-0.750258,-1.074151,-0.624945,0.425339
2,35.22,218.77,06993X0226/MEXI_2,5.220795,45.895734,2020-01-01,5.106667,45.833000,196.0,NaN,...,0.570933,0.702470,-0.011466,0.472359,0.262211,0.080137,-0.609068,2.384162,1.253131,0.151224
3,34.20,499.85,06533X0070/F2,5.948977,46.201180,2020-01-01,5.843000,46.302833,1133.0,0.0,...,1.248752,-1.499055,-0.907296,1.905138,-1.290514,1.433205,0.322991,-1.166792,0.120681,1.171185
4,37.30,260.00,06518X0026/P2,5.313353,46.136402,2020-01-01,5.287667,46.204333,260.0,0.0,...,-0.816898,1.060810,-0.166557,0.476222,-0.784821,1.551038,-0.934637,-0.373700,1.738446,-0.279887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2829905,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,-0.102097,-1.889806,-0.091534,-2.007449,-0.650497,-0.845332,1.049283,-0.000842,-0.089950,0.198511
2829906,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,-0.099234,0.349052,-0.325877,-0.908888,-1.561056,-0.973515,-1.655042,-0.252136,-1.555054,-0.344500
2829907,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,-0.803995,0.380239,1.036666,1.434841,-0.240442,1.476019,0.733772,0.355829,-0.657119,-1.096937
2829908,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,0.356369,-0.114541,-0.065868,0.845814,-0.324220,-0.028920,0.114309,0.571637,0.072600,0.962865


In [103]:
df_cleaned.memory_usage().sum() / 1e9

np.float64(8.749632336)

In [104]:
for i, typei in enumerate(df_cleaned.dtypes):
    if typei not in ['float64', 'bool']:
        print(df_cleaned.columns[i], typei)

piezo_station_bss_code object
piezo_measurement_date datetime64[ns]
insee_%_agri object
insee_med_living_level object
insee_%_ind object
insee_%_const object
piezo_groundwater_level_category object
piezo_station_department_code_01 object
piezo_station_department_code_02 object
piezo_station_department_code_03 object
piezo_station_department_code_04 object
piezo_station_department_code_05 object
piezo_station_department_code_06 object
piezo_station_department_code_07 object
piezo_station_department_code_08 object
piezo_station_department_code_09 object
piezo_station_department_code_10 object
piezo_station_department_code_11 object
piezo_station_department_code_12 object
piezo_station_department_code_13 object
piezo_station_department_code_14 object
piezo_station_department_code_15 object
piezo_station_department_code_16 object
piezo_station_department_code_17 object
piezo_station_department_code_18 object
piezo_station_department_code_19 object
piezo_station_department_code_21 object
pi

In [105]:
df_cleaned['insee_%_agri'].unique()

array(['11.8', '0.6', '0', '1.5', '0.2', '6', '2.6', '23.3', '1.6', '6.2',
       '2.1', '0.3', '11.1', '54.7', '2.3', '0.1', '28.3', '14.3', '3.7',
       '0.8', '18.2', '11.6', '21.4', '5', '16.7', '80', '13.3', '0.7',
       '0.4', '26.1', '17.4', '9.8', '30.4', '2.2', '20', '5.5', '0.5',
       '40.6', '5.9', '10', '60', '3.6', '6.1', '87.5', '6.4', '1.4', '7',
       '6.6', '4.8', '0.9', '8.3', '20.2', '3.3', '12.7', '1.7', '41.4',
       '33.3', '9.2', '62.1', '28.6', 'N/A - division par 0', '71.4',
       '3.2', '3.8', '1.1', '1.8', '7.1', '23.1', '18.8', '75', '32.1',
       '12.5', '1.2', '17.9', '6.9', '26.7', '20.8', '8.4', '25', '13',
       '27.5', '30.8', '25.9', '21.2', '18.4', '21.7', '2.5', '5.8',
       '22.8', '22.3', '30.9', '17.6', '3.4', '2.4', '4.7', '6.3', '26.4',
       '4.3', '9.5', '5.7', '5.1', nan, '27', '50', '7.7', '6.5', '12.9',
       '15.9', '12', '15.8', '4.9', '35.7', '58.3', '8.9', '10.3', '30',
       '8.7', '19.4', '8.5', '2.9', '16.4', '65.6', '2

In [106]:
df_cleaned['insee_med_living_level'].unique()

array(['25250', '24660', '24890', '39700', '26180', '22000', '24750',
       '35280', '24700', '26440', '26690', '27430', '25770', '24110',
       '24480', '22270', '24530', '20440', '26580', '20260', '18580',
       '19780', '21000', '22610', '23020', '21480', '21900', '21550',
       '20470', 'N/A - résultat non disponible', '23230', '23420',
       '18000', '18840', '19850', '21120', '25300', '16180', '20590',
       '16460', '19290', '20880', '24860', '23900', '19800', '25260',
       '30400', '23530', '22740', '23280', '23270', '23520', '21680',
       '21020', '23500', '22250', '24180', '21010', '20830', '23540',
       '20720', '21530', '21250', '20730', '17170', '18140', '22920',
       '22100', '23800', '23060', '20900', '20920', '21460', '23050',
       '22080', '22180', '21660', '19900', '19360', '21030', '21360',
       '22300', '26050', '29030', '24760', '22120', '24360', '21570',
       '23460', '27110', '24210', '23480', '23940', '26260', '25230',
       '29450', '25760'

In [107]:
df_cleaned['insee_%_ind'].unique()

array(['2.9', '44.5', '8.4', '2.4', '21.5', '3.3', '51.4', '3.4', '74.8',
       '1.2', '15.3', '11.1', '50.4', '2.2', '14.1', '22.9', '16.4', '0',
       '19', '21.4', '26.4', '72', '11.6', '5.4', '83', '15', '17.5',
       '50.7', '7.1', '54.4', '46.4', '8.8', '42.9', '20', '9.1', '15.1',
       '44.9', '3.1', '17.8', '9.8', '25', '53.6', '8.1', '17.2', '24.4',
       '24.3', '10', '18.4', '20.3', '18.8', '26.8', '8.3', '6.1', '42.6',
       '18.6', '8.6', '3.6', '49.2', '11.7', '4.1', '35.8', '5.3', '5.2',
       '13.4', '10.5', '60.8', '31', '1.8', '57.1', '3.9', '83.6', '14.3',
       'N/A - division par 0', '1.4', '40.4', '40.6', '34.5', '8.9',
       '4.2', '8.5', '15.7', '29.6', '13.1', '13', '7.2', '44.1', '6.4',
       '14.8', '1.5', '52.2', '7.5', '3.8', '14.9', '5.9', '53.1', '5',
       '8.7', '11', '11.2', '2.3', '19.5', '22.1', '9.2', '21.9', '12',
       '3.5', '11.3', '4.4', '16', '18.3', nan, '22.4', '73.5', '7.4',
       '33.8', '12.9', '0.6', '20.9', '9.7', '7.7', '

In [108]:
df_cleaned['insee_%_const'].unique()

array(['16.2', '11', '7.8', '5.2', '9.8', '8.4', '0.6', '5.1', '6.7',
       '21.4', '1.1', '52.1', '8.2', '5.6', '5.8', '34.4', '9', '17.4',
       '44.4', '17', '3.4', '5', '1.3', '0', '42.9', '1.6', '6', '63.6',
       '18.2', '26.1', '40', '3.5', '6.4', '30', '12.5', '13.3', '0.7',
       '14.5', '11.9', '9.2', '13', '4.3', '62.7', '56.5', '10.4', '2.2',
       '30.9', '3.9', '6.2', '45.9', '17.9', '21.7', '15.2', '34.8',
       '20.5', '19', '18.1', '8', '14.4', '23.9', '25', '10.3', '13.2',
       '5.3', '17.5', '10', '14.9', '10.1', '3.1', '8.8', '12.3', '9.5',
       '2', '24.6', '0.2', '10.7', '14.3', '0.5', 'N/A - division par 0',
       '75', '5.9', '1.9', '3', '33.8', '2.7', '3.7', '2.6', '61.5',
       '37.5', '2.1', '11.1', '43.6', '62.1', '8.3', '5.7', '53.8',
       '25.6', '4.5', '24.5', '16.7', '7.5', '8.7', '50', '2.4', '6.1',
       '1.8', '2.9', '7.1', '8.5', '4.8', '7.4', '5.4', '3.3', '10.9',
       '4', '4.7', '6.5', '18.5', '15.7', '28.9', '4.4', '6.3', '4.9',


In [109]:
df_cleaned['insee_%_agri'] = df_cleaned['insee_%_agri'].apply(lambda x: 'nan' if x=='N/A - division par 0' else x).astype(float)
df_cleaned['insee_med_living_level'] = df_cleaned['insee_med_living_level'].apply(lambda x: 'nan' if x=='N/A - résultat non disponible' else x).astype(float)
df_cleaned['insee_%_ind'] = df_cleaned['insee_%_ind'].apply(lambda x: 'nan' if x=='N/A - division par 0' else x).astype(float)
df_cleaned['insee_%_const'] = df_cleaned['insee_%_const'].apply(lambda x: 'nan' if x=='N/A - division par 0' else x).astype(float)


In [110]:
for i, typei in enumerate(df_cleaned.dtypes):
    if typei not in ['float64', 'bool']:
        print(df_cleaned.columns[i], typei)

piezo_station_bss_code object
piezo_measurement_date datetime64[ns]
piezo_groundwater_level_category object
piezo_station_department_code_01 object
piezo_station_department_code_02 object
piezo_station_department_code_03 object
piezo_station_department_code_04 object
piezo_station_department_code_05 object
piezo_station_department_code_06 object
piezo_station_department_code_07 object
piezo_station_department_code_08 object
piezo_station_department_code_09 object
piezo_station_department_code_10 object
piezo_station_department_code_11 object
piezo_station_department_code_12 object
piezo_station_department_code_13 object
piezo_station_department_code_14 object
piezo_station_department_code_15 object
piezo_station_department_code_16 object
piezo_station_department_code_17 object
piezo_station_department_code_18 object
piezo_station_department_code_19 object
piezo_station_department_code_21 object
piezo_station_department_code_22 object
piezo_station_department_code_23 object
piezo_statio

# Imputation géographique des valeurs manquantes avec la latitude et la longitude

In [111]:
panel = df[['piezo_station_bss_code', 'piezo_measurement_date', 'piezo_groundwater_level_category']].copy()
panel = panel.groupby(['piezo_station_bss_code', 'piezo_measurement_date']).sum()
panel

piezo_groundwater_level_category
piezo_station_bss_code piezo_measurement_date                                 
00035X0210/PZO1        2020-01-01                                         High
                       2020-01-02                                         High
                       2020-01-03                                         High
                       2020-01-04                                      Average
                       2020-01-05                                      Average
...                                                                        ...
BSS004KHTH/X           2022-04-17                                     Very Low
                       2022-04-18                                     Very Low
                       2022-04-19                                     Very Low
                       2022-04-20                                     Very Low
                       2022-04-21                                     Very Low

[2830072 rows x 1 columns]

In [112]:
individuals = panel.index.get_level_values(0).unique()
time_values = panel.index.get_level_values(1).unique()

# Create a complete index with all combinations of individuals and time
complete_index = pd.MultiIndex.from_product([individuals, time_values], 
                                            names=['piezo_station_bss_code', 'piezo_measurement_date'])

# Reindex the original DataFrame to include all combinations
panel = panel.reindex(complete_index)

In [113]:
panel

piezo_groundwater_level_category
piezo_station_bss_code piezo_measurement_date                                 
00035X0210/PZO1        2020-01-01                                         High
                       2020-01-02                                         High
                       2020-01-03                                         High
                       2020-01-04                                      Average
                       2020-01-05                                      Average
...                                                                        ...
BSS004KHTH/X           2023-05-27                                          NaN
                       2023-05-28                                          NaN
                       2023-05-29                                          NaN
                       2023-05-30                                          NaN
                       2023-05-31                                          NaN

[3207375 rows x 1 columns]

In [114]:
panel.reset_index().isna().sum()

piezo_station_bss_code                   0
piezo_measurement_date                   0
piezo_groundwater_level_category    377303
dtype: int64

In [115]:
# on prend le plus proche voisin et on remplace

In [116]:
stations_lat_lon = df[['piezo_station_bss_code', 'piezo_station_latitude', 'piezo_station_longitude']].drop_duplicates().set_index('piezo_station_bss_code').copy()
stations_lat_lon

,piezo_station_latitude,piezo_station_longitude
piezo_station_bss_code,,
07004X0046/D6-20,45.828148,5.748241
06754X0077/F1,46.028102,5.356637
06993X0226/MEXI_2,45.895734,5.220795
06533X0070/F2,46.201180,5.948977
06518X0026/P2,46.136402,5.313353
...,...,...
02691X0006/F3,48.529391,6.541648
08042X0034/F2,44.952451,-0.236980
08042X0042/F3,44.952451,-0.236980


In [117]:
def calculate_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great-circle distance between two points on Earth using the Haversine formula.
    
    Parameters:
    lat1, lon1: Latitude and longitude of the first point (in decimal degrees)
    lat2, lon2: Latitude and longitude of the second point (in decimal degrees)
    
    Returns:
    Distance between the two points in the specified unit
    """
    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371
    
    # Convert latitude and longitude to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Calculate the great-circle distance
    a = (math.sin(dlat/2)**2 + 
         math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2)
    central_angle = 2 * math.asin(math.sqrt(a))
    
    # Calculate distance based on the chosen unit
    distance = central_angle * EARTH_RADIUS_KM
    
    return distance

In [118]:
stations_distances = np.zeros((100, 100))
for i in range(0, 100):
    for j in range(0, i):
        stations_distances[i, j] = calculate_distance(stations_lat_lon.iloc[i, 0], stations_lat_lon.iloc[i, 1], stations_lat_lon.iloc[j, 0], stations_lat_lon.iloc[j, 1])
for i in range(0, 100):
    for j in range(i, 100):
        stations_distances[i, j] = stations_distances[j, i]
for i in range(0, 100):
    stations_distances[i, i] = np.inf

In [119]:
stations_distances

array([[         inf,  37.57238558,  41.52835062, ..., 134.45422638,
        134.59041776, 134.63292326],
       [ 37.57238558,          inf,  18.08000188, ..., 166.25890763,
        166.21680626, 165.96424998],
       [ 41.52835062,  18.08000188,          inf, ..., 158.24893028,
        158.07514436, 157.61157509],
       ...,
       [134.45422638, 166.25890763, 158.24893028, ...,          inf,
          1.32428428,   3.47383407],
       [134.59041776, 166.21680626, 158.07514436, ...,   1.32428428,
                 inf,   2.15334114],
       [134.63292326, 165.96424998, 157.61157509, ...,   3.47383407,
          2.15334114,          inf]])

In [120]:
temp = panel.isna().groupby(level=0).sum() == 0
temp

,piezo_groundwater_level_category
piezo_station_bss_code,
00035X0210/PZO1,True
00061X0117/PZ1,True
00065X0148/P,False
00067X0183/PZ1,False
00068X0010/F295,False
...,...
BSS004EPHV/X,True
BSS004HNDZ/X,False
BSS004KHNR/X,True


In [121]:
# je veux un dictionnaire de stations_incompletes à stations_completes
stations_completes = temp[temp['piezo_groundwater_level_category'] == True].index
stations_incompletes = temp[temp['piezo_groundwater_level_category'] == False].index
stations_distances = np.zeros((len(stations_incompletes), len(stations_completes)))
for i in tqdm(range(0, len(stations_incompletes))):
    for j in range(0, len(stations_completes)):
        stations_distances[i, j] = calculate_distance(stations_lat_lon.loc[stations_incompletes[i], 'piezo_station_latitude'], stations_lat_lon.loc[stations_incompletes[i], 'piezo_station_longitude'], stations_lat_lon.loc[stations_completes[j], 'piezo_station_latitude'], stations_lat_lon.loc[stations_completes[j], 'piezo_station_longitude'])

station_incomplete_to_station_complete = {}
for i in range(0, len(stations_incompletes)):
    station_incomplete_to_station_complete[stations_incompletes[i]] = stations_completes[np.argmin(stations_distances[i, :])]

print(station_incomplete_to_station_complete)
    

100%|██████████| 1398/1398 [00:51<00:00, 27.37it/s]

{'00065X0148/P': '00061X0117/PZ1', '00067X0183/PZ1': '00113X0002/P1', '00068X0010/F295': '00068X0147/PZ5', '00122X0462/PZ': '00125X0010/S1', '00146X2040/PZ5': '00146B0007/F1', '00147D0218/P1': '00147X3381/PZ6', '00182X0031/P': '00182X0010/P1', '00197X0049/F2': '00198X0123/P1', '00202X0463/PZ4': '00203A0359/PZ2', '00204X0260/PZ1': '00203A0359/PZ2', '00213X0002/F1': '00216X0073/PZ3', '00275X0243/PZ1': '00275X0005/P1', '00276X0030/F1': '00275X0005/P1', '00276X0114/PZ3': '00275X0005/P1', '00276X0115/PZ4': '00275X0005/P1', '00277X0157/PZ5': '00281X0004/F1', '00277X0158/PZ6': '00281X0004/F1', '00278X0132/PZ7': '00281X0004/F1', '00278X0133/PZ8': '00281X0004/F1', '00285X0477/PZ9B': '00281X0004/F1', '00285X0478/PZ10': '00286X0405/PZST2', '00326X0018/P': '00327X0059/P11', '00346X0011/S1': '00341X0050/PZ1', '00347X0002/P': '00461X0040/FE2', '00376X0210/PZ1': '00377X0051/S1', '00378X0162/PZCAT4': '00377X0051/S1', '00383X0229/PZSP6': '00387X0184/PZCC1', '00441X0007/P': '00445X0006/P', '00442X0050/P

In [ ]:
for station in tqdm(stations_incompletes):
    station_mask = panel.index.get_level_values(0) == station
    # prendre le plus proche voisin parmi les séries complètes
    replacement_series = panel.xs(station_incomplete_to_station_complete[station], level='piezo_station_bss_code')
    # compléter les valeurs manquantes
    panel.loc[station_mask] = panel.loc[station_mask].fillna(replacement_series)


 31%|███       | 435/1398 [01:22<02:46,  5.80it/s]

In [ ]:
panel.isna().sum()

In [ ]:
stations_subset = np.unique(df_cleaned['piezo_station_bss_code'])

In [ ]:
panels = [panel.xs(station, level='piezo_station_bss_code').sort_index() for station in stations_subset]
panels[0]

In [ ]:
ete_2021 = [f'2021-{str(month).zfill(2)}-{str(day).zfill(2)}' for month in range(6, 10) for day in range(1, 32)]
ete_2021.remove('2021-06-31')
ete_2021.remove('2021-09-31')
ete_2021

In [ ]:
ete_2022 = [f'2022-{str(month).zfill(2)}-{str(day).zfill(2)}' for month in range(6, 10) for day in range(1, 32)]
ete_2022.remove('2022-06-31')
ete_2022.remove('2022-09-31')
ete_2022

In [ ]:
# C'est peut-être mieux d'utiliser les valeurs qu'on a prédites pour l'été 2022

In [ ]:
for ix, pl in enumerate(panels):
    new_rows = pl.loc[ete_2021].sort_index().copy()
    new_rows.index = ete_2022
    new_rows.index.name = 'piezo_measurement_date'
    panels[ix] = pd.concat([pl, new_rows], ignore_index=False)

In [ ]:
panels[0]

In [ ]:
panels_with_lags = [pd.concat([pl.shift(i) for i in range(1, 365+1)], axis=1) for pl in tqdm(panels)]
len(panels_with_lags)

In [ ]:
panels_with_lags

In [ ]:
for i in range(len(panels_with_lags)):
    panels_with_lags[i]['piezo_station_bss_code'] = stations_subset[i]
    panels_with_lags[i] = panels_with_lags[i].reset_index().set_index(['piezo_station_bss_code', 'piezo_measurement_date'])

In [ ]:
panels_with_lags[0]

In [ ]:
df_with_lags = pd.concat(panels_with_lags)
df_with_lags

In [ ]:
df_with_lags.columns = [f'lag_{i+1}' for i in range(365)]
df_with_lags

In [ ]:
df_cleaned

In [ ]:
del df

In [ ]:
df_with_lags = df_with_lags.reset_index()
df_with_lags

In [ ]:
df_with_lags['piezo_measurement_date'] = pd.to_datetime(df_with_lags['piezo_measurement_date'])
df_with_lags['piezo_measurement_date']

In [ ]:
df_cleaned = pd.merge(df_cleaned, df_with_lags, how='left', on=['piezo_station_bss_code', 'piezo_measurement_date']).sort_values(by=['piezo_station_bss_code', 'piezo_measurement_date'])
df_cleaned = df_cleaned.set_index(['piezo_station_bss_code', 'piezo_measurement_date'])
df_cleaned

In [ ]:
mapping_dict = {
    'Very Low': 0,
    'Low': 0.25,
    'Average': 0.5,
    'High': 0.75,
    'Very High':1,
    'None':999
}
for i in range(365):
    df_cleaned[f'lag_{i+1}'] = df_cleaned[f'lag_{i+1}'].map(mapping_dict)
df_cleaned

In [ ]:
df_cleaned.memory_usage().sum() / 1e9

# XGBoost

In [ ]:
df_cleaned = df_cleaned.reset_index()

In [ ]:
# Prédire de février à mai 2022 # il faudra faire attention car erreurs dans les lags après l'été 2022 (car on remonte à plus d'un an vu qu'il n'y a pas les valeurs de l'été)

In [ ]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df_cleaned['piezo_groundwater_level_category'])
y

In [ ]:
X_train = df_cleaned [df_cleaned['piezo_measurement_date'] < '2022-02-01'].drop(['piezo_groundwater_level_category', 'piezo_station_bss_code', 'piezo_measurement_date'], axis=1)
y_train = y [df_cleaned['piezo_measurement_date'] < '2022-02-01']
X_test = df_cleaned [(df_cleaned['piezo_measurement_date'] >= '2022-02-01') & (df_cleaned['piezo_measurement_date'] < '2022-06-01')].drop(['piezo_groundwater_level_category', 'piezo_station_bss_code', 'piezo_measurement_date'], axis=1)
dates_test = df_cleaned [(df_cleaned['piezo_measurement_date'] >= '2022-02-01') & (df_cleaned['piezo_measurement_date'] < '2022-06-01')]['piezo_measurement_date']
stations_test = df_cleaned [(df_cleaned['piezo_measurement_date'] >= '2022-02-01') & (df_cleaned['piezo_measurement_date'] < '2022-06-01')]['piezo_station_bss_code']
y_test = y [(df_cleaned['piezo_measurement_date'] >= '2022-02-01') & (df_cleaned['piezo_measurement_date'] < '2022-06-01')]


In [ ]:
stations_test.value_counts()

In [ ]:
value_map = {0: 0.5,
 1: 0.75,
 2: 0.25,
 3: 1.,
 4: 0.}

In [ ]:
def map_values(x):
        return value_map.get(x, x)  # Use x if x is not in the dictionary

In [ ]:
dates_pred = [f'2022-{str(m).zfill(2)}-{str(i).zfill(2)}' for m in range(2, 6) for i in range(1, 32)]
dates_pred.remove('2022-02-29')
dates_pred.remove('2022-02-30')
dates_pred.remove('2022-02-31')
dates_pred.remove('2022-04-31')
dates_pred

In [ ]:
# mask forbidden lags
for i, date in enumerate(dates_pred): 
    print(date)
    for j in range(i):
        # print(j)
        X_test[dates_test == date][f'lag_{j+1}'] = None

In [ ]:
def replace_with_neighbor_value(row):
    if np.isnan(row[0]):
        return pd.Series({'piezo_station_bss_code': row['piezo_station_bss_code'], 0:y_pred_with_station.loc[station_incomplete_to_station_complete[row['piezo_station_bss_code']], 0]})
    return row

In [ ]:
pd.options.mode.chained_assignment = None  # Options: 'warn', 'raise', None

In [ ]:
model = xgb.XGBClassifier(   # voir pour faire un regressor plutôt
    objective='multi:softmax',  
    max_depth=12,                
    learning_rate=0.1,          
    n_estimators=100,           
    verbosity=1,               
    # random_state=42             
)

model.fit(X_train, y_train, verbose=True)

importance = model.feature_importances_
important_features = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': importance
}).sort_values(by='Importance', ascending=False)

print(important_features.head(10)) 


y_pred_all_test_set_with_all_stations = pd.DataFrame(columns=['piezo_station_bss_code', 'piezo_measurement_date', 'piezo_groundwater_level_category'])

for ixdate, date in enumerate(dates_pred):
    y_pred = model.predict(X_test[dates_test == date])
    
    accuracy = accuracy_score(y_test[dates_test == date], y_pred)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    
    # Use numpy.vectorize to apply the function to the array
    vectorized_map = np.vectorize(map_values)
    y_pred = vectorized_map(y_pred)

    y_pred_with_station = pd.DataFrame(data=y_pred, index=stations_test[dates_test == date])
    y_pred_with_station_all = y_pred_with_station.reindex(stations_subset).reset_index()
    y_pred_with_station_all = y_pred_with_station_all.apply(replace_with_neighbor_value, axis=1).set_index('piezo_station_bss_code')
    
    y_pred_all_test_set_with_all_stations = pd.concat([y_pred_all_test_set_with_all_stations, pd.DataFrame(data=list(zip(y_pred_with_station_all.index, [date]*len(y_pred_with_station_all), y_pred_with_station_all[0])), columns=['piezo_station_bss_code', 'piezo_measurement_date', 'piezo_groundwater_level_category'])])

    for i, datei in enumerate(dates_pred[ixdate+1:]):
        
        print(datei, i+1)
        X_test[dates_test == datei][f'lag_{i+1}'] = y_pred_with_station_all.loc[stations_test[dates_test == datei].values].copy() # par contre il y a des valeurs manquantes donc il faut à nouveau utiliser la stratégie du plus proche voisin


In [ ]:
panel.update(y_pred_all_test_set_with_all_stations.sort_values(by=['piezo_station_bss_code', 'piezo_measurement_date']).set_index(['piezo_station_bss_code', 'piezo_measurement_date']))

In [ ]:
panels = [panel.xs(station, level='piezo_station_bss_code').sort_index() for station in stations_subset]
panels[0]

In [ ]:
panels_with_lags = [pd.concat([pl.shift(i) for i in range(1, 365+1)], axis=1) for pl in tqdm(panels)]
len(panels_with_lags)

In [ ]:
for i in range(len(panels_with_lags)):
    panels_with_lags[i]['piezo_station_bss_code'] = stations_subset[i]
    panels_with_lags[i] = panels_with_lags[i].reset_index().set_index(['piezo_station_bss_code', 'piezo_measurement_date'])

In [ ]:
df_with_lags = pd.concat(panels_with_lags)
df_with_lags

In [ ]:
df_with_lags.columns = [f'lag_{i+1}' for i in range(365)]
df_with_lags

In [ ]:
df_with_lags = df_with_lags.reset_index()
df_with_lags

In [ ]:
df_with_lags['piezo_measurement_date'] = pd.to_datetime(df_with_lags['piezo_measurement_date'])
df_with_lags['piezo_measurement_date']

In [ ]:
keep = df_cleaned.columns[:-365]

In [ ]:
df_cleaned = pd.merge(df_cleaned[keep], df_with_lags, how='left', on=['piezo_station_bss_code', 'piezo_measurement_date']).sort_values(by=['piezo_station_bss_code', 'piezo_measurement_date'])
df_cleaned = df_cleaned.set_index(['piezo_station_bss_code', 'piezo_measurement_date'])
df_cleaned

In [ ]:
for i in range(365):
    df_cleaned[f'lag_{i+1}'] = df_cleaned[f'lag_{i+1}'].map(mapping_dict)
df_cleaned

In [ ]:
df_cleaned.memory_usage().sum() / 1e9

In [ ]:
df_cleaned = df_cleaned.reset_index()

In [ ]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df_cleaned['piezo_groundwater_level_category'])
y

In [ ]:
X_test = df_cleaned [(df_cleaned['piezo_measurement_date'] >= '2023-02-01') & (df_cleaned['piezo_measurement_date'] < '2023-06-01')].drop(['piezo_groundwater_level_category', 'piezo_station_bss_code', 'piezo_measurement_date'], axis=1)
dates_test = df_cleaned [(df_cleaned['piezo_measurement_date'] >= '2023-02-01') & (df_cleaned['piezo_measurement_date'] < '2023-06-01')]['piezo_measurement_date']
stations_test = df_cleaned [(df_cleaned['piezo_measurement_date'] >= '2023-02-01') & (df_cleaned['piezo_measurement_date'] < '2023-06-01')]['piezo_station_bss_code']
y_test = y [(df_cleaned['piezo_measurement_date'] >= '2023-02-01') & (df_cleaned['piezo_measurement_date'] < '2023-06-01')]


In [ ]:
y_test

In [ ]:
stations_test.value_counts()

In [ ]:
value_map = {0: 0.5,
 1: 0.75,
 2: 0.25,
 3: 1.,
 4: 0.}

In [ ]:
def map_values(x):
        return value_map.get(x, x)  # Use x if x is not in the dictionary

In [ ]:
# mask forbidden lags
dates_pred = [f'2023-{str(m).zfill(2)}-{str(i).zfill(2)}' for m in range(2, 6) for i in range(1, 32)]
dates_pred.remove('2023-02-29')
dates_pred.remove('2023-02-30')
dates_pred.remove('2023-02-31')
dates_pred.remove('2023-04-31')
for i, date in enumerate(dates_pred): 
    print(date)
    for j in range(i):
        print(j)
        X_test[dates_test == date][f'lag_{j+1}'] = None

In [ ]:
def replace_with_neighbor_value(row):
    if np.isnan(row[0]):
        return pd.Series({'piezo_station_bss_code': row['piezo_station_bss_code'], 0:y_pred_with_station.loc[station_incomplete_to_station_complete[row['piezo_station_bss_code']], 0]})
    return row

In [ ]:
# y_pred_all_test_set_with_all_stations = pd.DataFrame(columns=['piezo_station_bss_code', 'piezo_measurement_date', 'piezo_groundwater_level_category'])

for ixdate, date in enumerate(dates_pred):
    y_pred = model.predict(X_test[dates_test == date])
    
    accuracy = accuracy_score(y_test[dates_test == date], y_pred)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    
    # Use numpy.vectorize to apply the function to the array
    vectorized_map = np.vectorize(map_values)
    y_pred = vectorized_map(y_pred)

    y_pred_with_station = pd.DataFrame(data=y_pred, index=stations_test[dates_test == date])
    y_pred_with_station_all = y_pred_with_station.reindex(stations_subset).reset_index()
    y_pred_with_station_all = y_pred_with_station_all.apply(replace_with_neighbor_value, axis=1).set_index('piezo_station_bss_code')
    
    y_pred_all_test_set_with_all_stations = pd.concat([y_pred_all_test_set_with_all_stations, pd.DataFrame(data=list(zip(y_pred_with_station_all.index, [date]*len(y_pred_with_station_all), y_pred_with_station_all[0])), columns=['piezo_station_bss_code', 'piezo_measurement_date', 'piezo_groundwater_level_category'])])

    for i, datei in enumerate(dates_pred[ixdate+1:]):
        
        print(datei, i+1)
        X_test[dates_test == datei][f'lag_{i+1}'] = y_pred_with_station_all.loc[stations_test[dates_test == datei].values].copy() # par contre il y a des valeurs manquantes donc il faut à nouveau utiliser la stratégie du plus proche voisin


In [ ]:
# Save model
model.save_model('xgboost_model.json')  # JSON format
# model.save_model('xgboost_model.ubj')   # Universal Binary JSON format

In [ ]:
# Load model later
# loaded_model = xgb.Booster()
# loaded_model.load_model('xgboost_model.json')

In [ ]:
# score final

In [ ]:
y_pred_all_test_set_with_all_stations

In [ ]:
faire un merge avec le dataset de base et regarder l'accuracy

In [ ]:
df = pd.read_csv('X_train_Hi5.csv',  usecols=['row_index', 'piezo_station_bss_code', 'piezo_measurement_date', 'piezo_groundwater_level_category']).set_index('row_index')
df.head()

In [ ]:
pd.merge(df, y_pred_all_test_set_with_all_stations, on=['piezo_station_bss_code', 'piezo_measurement_date'])